In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [2]:
jan = pd.read_csv('Sales_January_2019.csv')
feb = pd.read_csv('Sales_February_2019.csv')
march = pd.read_csv('Sales_March_2019.csv')
april = pd.read_csv('Sales_April_2019.csv')
may = pd.read_csv('Sales_May_2019.csv')
june = pd.read_csv('Sales_June_2019.csv')
july = pd.read_csv('Sales_July_2019.csv')
aug = pd.read_csv('Sales_August_2019.csv')
sep = pd.read_csv('Sales_September_2019.csv')
october = pd.read_csv('Sales_October_2019.csv')
nov = pd.read_csv('Sales_November_2019.csv')
dec = pd.read_csv('Sales_December_2019.csv')

In [3]:
data = pd.concat([jan,feb,march,april,may,june,july,aug,sep,october,nov,dec],ignore_index=True)
data.dropna(how='all',inplace=True)

## 1. What was the Best Month for Sales? How much was Earned that Month?

In [4]:
data.head(3)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"


In [5]:
data = data[data['Order ID'] != 'Order ID']
data.reset_index(drop=True,inplace=True)

In [6]:
data['Order Date'] = pd.to_datetime(data['Order Date'],format= "%m/%d/%y %H:%M")

In [7]:
data['Order ID'] = data['Order ID'].astype(int)
data['Quantity Ordered'] = data['Quantity Ordered'].astype(int)
data['Price Each'] = data['Price Each'].astype(float)
month = data['Order Date'].dt.month
date = data['Order Date'].dt.date
year = data['Order Date'].dt.year
time = data['Order Date'].dt.time

In [8]:
data['Total_Revenue'] = data['Quantity Ordered'] * data['Price Each']

In [9]:
top_sales_month = data.groupby(month)['Total_Revenue'].sum().sort_values(ascending=False).reset_index()
top_sales_month.columns = ['Month','Total_Revenue']
top_sales_month.head(3)

,Month,Total_Revenue
0,12,4613443.34
1,10,3736726.88
2,4,3390670.24


In [10]:
fig = px.bar(top_sales_month,x='Month',y='Total_Revenue',title='Total Revenue by Month',
            color='Total_Revenue',color_continuous_scale='ylorrd',
            text_auto='0.2s'
            )
fig.update_layout(xaxis_title='Month',yaxis_title='Sales')
fig.show()

## 2. What City Sold the Most Products?

In [11]:
data['City'] = data['Purchase Address'].apply(lambda x: x.split(',')[1] + f" ({x.split(',')[2].split(" ")[1]})")
data['City']

0                 Boston (MA)
1               Portland (OR)
2          San Francisco (CA)
3            Los Angeles (CA)
4                 Austin (TX)
                 ...         
185945     San Francisco (CA)
185946       Los Angeles (CA)
185947           Seattle (WA)
185948            Dallas (TX)
185949       Los Angeles (CA)
Name: City, Length: 185950, dtype: str

In [12]:
high_sales_city = data.groupby('City')['Quantity Ordered'].sum().sort_values(ascending=False).reset_index()
high_sales_city

,City,Quantity Ordered
0,San Francisco (CA),50239
1,Los Angeles (CA),33289
2,New York City (NY),27932
3,Boston (MA),22528
4,Dallas (TX),16730
5,Atlanta (GA),16602
6,Seattle (WA),16553
7,Portland (OR),11303
8,Austin (TX),11153
9,Portland (ME),2750


In [13]:
fig = px.bar(high_sales_city,x='City',y='Quantity Ordered',title='Higest Sales in City',text_auto='0.2s',
        color='Quantity Ordered')
fig.update_layout(xaxis_title='Cities',yaxis_title='Quanity Ordered')
fig.show()


## 3. What Time should We Display Advertisements to Maximize the Likeklihood of Customer's Buying Product?

In [14]:
hour = data['Order Date'].dt.hour
minute = data['Order Date'].dt.minute

In [15]:
high_sales_hour = data.groupby(hour)['Quantity Ordered'].sum().reset_index()
high_sales_hour.columns = ['Hour','Quantity Ordered']

In [16]:
high_sales_hour.head(3)

,Hour,Quantity Ordered
0,0,4428
1,1,2619
2,2,1398


In [17]:
fig = px.line(high_sales_hour,x='Hour',y='Quantity Ordered',title='Higest Sales by Hour',markers=True)       
fig.update_layout(xaxis_title='Hour',yaxis_title='Quanity Ordered')
fig.show()

## 4. What Produsts are Most Often Sold Together?

In [18]:
data['Sold_together'] = data.groupby('Order ID')['Product'].transform(lambda x: ', '.join(x))

In [19]:
data = data.drop_duplicates(subset=['Order ID','Sold_together'])

In [20]:
most_products_sold_together = data.groupby('Sold_together')['Quantity Ordered'].sum().sort_values(ascending=False).reset_index()
most_products_sold_together.head(3)

,Sold_together,Quantity Ordered
0,AAA Batteries (4-pack),29766
1,AA Batteries (4-pack),26622
2,USB-C Charging Cable,21683


In [28]:
fig = go.Figure(
    go.Bar(
        x=most_products_sold_together['Sold_together'].head(10),
        y=most_products_sold_together['Quantity Ordered'].head(10),
        text=most_products_sold_together['Quantity Ordered'].head(10),
        texttemplate='<b>%{text:,}</b>',
        textposition='outside',
        marker=dict(
            color=most_products_sold_together['Quantity Ordered'].head(10),
            colorscale='ylorrd'),
        hovertemplate=(
            '<b>%{x}</b><br>'
            'Quantity Ordered: %{y:,}'
            '<extra></extra>')))
fig.update_layout(
    width=1200,
    height=650,
    title=dict(
        text='📦 Most Products Sold Together',
        x=0.5),
    xaxis_title='<b>Products Sold Together</b>',
    yaxis_title='<b>Quantity Ordered</b>')
fig.show()

## 5. What Product Sold the Most? Why do You Think It Sold the Most?

In [22]:
most_sold_products = data.groupby('Product')['Quantity Ordered'].sum().reset_index()
most_sold_products.head(5)

,Product,Quantity Ordered
0,20in Monitor,4049
1,27in 4K Gaming Monitor,6118
2,27in FHD Monitor,7393
3,34in Ultrawide Monitor,6073
4,AA Batteries (4-pack),27136


In [23]:
average_price_per_product = data.groupby('Product')['Price Each'].mean().reset_index()
average_price_per_product.head(3)

,Product,Price Each
0,20in Monitor,109.99
1,27in 4K Gaming Monitor,389.99
2,27in FHD Monitor,149.99


In [24]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=most_sold_products['Product'],
    y=most_sold_products['Quantity Ordered'],
    text=most_sold_products['Quantity Ordered'],
        texttemplate='<b>%{text:,}</b>',
        textposition='outside',
     marker=dict(
            color=most_products_sold_together['Quantity Ordered']),
    yaxis='y1'))
fig.add_trace(go.Scatter(
    x=average_price_per_product['Product'],
    y=average_price_per_product['Price Each'],
     mode='lines+markers',
     yaxis='y2'))
fig.update_layout(
    width=1200,
    height=500,
    title=dict(
        text='📊 Quantity Sold vs Average Price per Product',
        x=0.5
    ),
    xaxis=dict(title='Product'),
    yaxis=dict(
        title='Quantity Ordered',
        showgrid=True,
        gridcolor='lightgray'
    ),
    yaxis2=dict(
        title='Average Price ($)',
        overlaying='y',
        side='right'
    ),
    template='plotly_white'
)
